In [1]:
from peewee import *


In [7]:
db_descriptor = 'dhblab_meta'
# db_descriptor = 'file:dhblab_meta?mode=memory&cache=shared'

In [8]:
db = SqliteDatabase(db_descriptor)

In [9]:
class Manipulator(Model):
    class Meta:
        database = db
    class Table:
        db_table = 'manipulator'
    name = CharField(unique=True)

class AMR(Model):
    class Meta:
        database = db
        db_table = 'amr'
    name = CharField()

class Aerocraft(Model):
    class Meta:
        database = db
        db_table = 'aerocraft'
    name = CharField()

In [10]:
db.create_tables([Manipulator, AMR, Aerocraft])
print(db.table_exists('aerocraft'))

True


In [11]:
a1 = Manipulator.create(name='1')
a1.save()

1

In [12]:
a1 = Manipulator.create(name='2')
a1.save()

1

In [13]:
db.close()

True